# The Battle of Neighborhoods

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>

How excited when the holiday season is coming, and you plan to have a wonderful trip with your family. And then the first problem you need take time to tackle is where to go. You may have many different options, the places you would like to visit, and they all look attractive. 

Sometimes it is quite hard to choose a desired one and convince your partner too.

An application that can compare the optional destinations and determine how similar or dissimilar they are would be useful for this case. Practically, the app can group the destinations into different categories, and thus it will help you make a decision based on the grouped result. 

E.g. you may have 5 possible destinations to travel: Tenerife, Santorini, Maldives, Phuket, Honolulu. They are all perfect islands for travel during vacation. It would be helpful to find out whether those islands are similar or dissimilar. E.g. If Maldives and Phuket fall into one group, but they are in different groups than Tenerife and Santorini, for the users who want to experience two different kinds of islands, they can choose one of them from Maldives and Phuket and another from Tenerife and Santorini.


## Data <a name="data"></a>

Found the neighborhood of each place.

For each of places, we can get the neighborhoods, longitude and latitude value of the neighborhoods from https://www.geonames.org/.
 
We can read the result table into Pandas dataframe.
The latitude and longitude values are in the format as 

N 36° 21' 29'', E 25° 26' 46''
We use the following formula to translate the latitude and longitude value into decimal format

Decimal Degrees = degrees + (minutes/60) + (seconds/3600)


Then for each of places, we can get one neighborhood dataframe and then we merge them into a big one as ‘Neighborhoods’. That is our data we use for analysis.


In [1]:
import pandas as pd
import numpy as np
import re 

Define function to convert latitude and longitude to decimal.

In [2]:
def latlongconverter(data):
    values = re.findall(r'\d+', data)
    result = (int(values[0]) + int(values[1]) / 60 + int(values[2]) / 3600) * (1 if data.startswith('N') or data.startswith('E') else -1)
    return result

Define function to get neighborhood of one place by given the link to geonames search results

In [3]:
def createDataFrameFromLink(link):
    dfs = pd.read_html(link)
    dfsnew = dfs[2].drop([0])
    dfsnew = dfsnew.drop([0], axis=1)
    new_header = dfsnew.iloc[0] #grab the first row for the header
    dfsnew = dfsnew[1:] #take the data from the third row
    dfsnew.columns = new_header #set the header row as the df header
    dfsnew = dfsnew.dropna()
    place_data = dfsnew.reset_index(drop=True)
    place_data = place_data.rename(columns={"Name": "Neighborhood"})
    place_data['Latitude'] = place_data['Latitude'].apply(latlongconverter)
    place_data['Longitude'] = place_data['Longitude'].apply(latlongconverter)
    place_data['Neighborhood'] = place_data['Neighborhood'].apply(lambda x : x.split(',')[0])
    return place_data

Get data for each of place

In [4]:
santorini_data = createDataFrameFromLink('https://www.geonames.org/advanced-search.html?q=Santorini&country=GR&featureClass=P&continentCode=EU')

In [116]:
santorini_data.shape

(37, 5)

In [5]:
tenerife_data = createDataFrameFromLink('https://www.geonames.org/advanced-search.html?q=Tenerife&country=ES&featureClass=P&continentCode=EU')

In [112]:
tenerife_data.shape

(50, 5)

In [6]:
maldives_data = createDataFrameFromLink('https://www.geonames.org/advanced-search.html?q=Maldives&country=MV&featureClass=P&continentCode=AS&startRow=0')
maldives_data.shape

(50, 5)

In [7]:
phuket_data = createDataFrameFromLink('https://www.geonames.org/advanced-search.html?q=Phuket&country=TH&featureClass=P&continentCode=AS')
phuket_data.shape

(50, 5)

In [8]:
honolulu_data = createDataFrameFromLink('https://www.geonames.org/advanced-search.html?q=honolulu&country=US&featureClass=P&continentCode=NA')
honolulu_data.shape

(50, 5)

Merge the data of each place into one dataframe

In [9]:
place_data = pd.concat([santorini_data, tenerife_data, maldives_data, phuket_data, honolulu_data], axis=0)

In [10]:
place_data.shape

(237, 5)

In [121]:
place_data.tail()

1,Neighborhood,Country,Feature class,Latitude,Longitude
45,Hauâula Hau'ula,"United States, HawaiiHonolulu","populated placepopulation 4,148, elevation 1m",21.607500,-157.908611
46,Heâeia He'eia,"United States, HawaiiHonolulu","populated placepopulation 4,963, elevation 28m",21.430278,-157.816111
47,âAiea 'Aiea,"United States, HawaiiHonolulu","populated placepopulation 9,338, elevation 30m",21.382222,-157.933611
48,âÄhuimanu 'Ahuimanu,"United States, HawaiiHonolulu","populated placepopulation 8,810, elevation 25m",21.444722,-157.837778
49,Iroquois Point,"United States, HawaiiHonolulu","populated placepopulation 3,374, elevation 2m",21.327500,-157.982778


Show data in map

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [12]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [13]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         704 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.0-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

In [14]:
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [16]:
address = 'Honolulu'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Honolulu are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Honolulu are 21.304547, -157.8556764.


In [17]:
# create map of Maldives using latitude and longitude values
map_places = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(place_data['Latitude'], place_data['Longitude'], place_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_places)  
    
map_places

## Methodology <a name="methodology"></a>

We would like to compare those places and know if they are similar. The basic idea is we run the k-means clusters on the features of the places, if two places are in the same group, we consider them as similar.

We do the following of steps to get the features of places.


Get venues for each neighborhood
For each of neighborhood in the dataframe ‘Neighborhoods’, we call the foursquare api:
https://api.foursquare.com/v2/venues/search?client_id={{client_id}}&client_secret={{client_secret}}&v={{v}}&ll=&intent=browse&radius=10000&limit=100
The longitude and latitude of neighborhoods are got from the previous step, and the radius is using 5 km. The limit of result is 100. 

The result dataframe would be the dataframe of neighborhoods’ venues that that a typical user is likely to checkin to at the provided

In [18]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200105' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XCCTRA0MPPISQMTGRR54VBTS4AU301WLROHIKUBLEZLPYQ5R
CLIENT_SECRET:4PRSIBBSFJYTCYIQ2QY1SLAJZKS1V1UHLQMV435NYP0SG5FY


Define function to getNearbyNenues for neighborhoods

In [19]:
LIMIT = 100 # result is 100 limit

radius = 5000 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
place_venues = getNearbyVenues(names=place_data['Neighborhood'],
                                   latitudes=place_data['Latitude'],
                                   longitudes=place_data['Longitude']
                                  )

In [ ]:
Thus we get 100 venues for each of neighborhoods and eventually we get a new big dataframe of neighborhood venues 

In [21]:
print(place_venues.shape)
place_venues.head()

(2665, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,EmporeÃ­o Emborio,36.358056,25.446111,Καστέλι Εμπορείου,36.358796,25.446271,Castle
1,EmporeÃ­o Emborio,36.358056,25.446111,Το καφενεδάκι του Εμπορείου,36.358054,25.445674,Café
2,EmporeÃ­o Emborio,36.358056,25.446111,Πλατεία Εμπορείου,36.356918,25.444290,Plaza
3,EmporeÃ­o Emborio,36.358056,25.446111,Σταύρος,36.355636,25.446680,Grilled Meat Restaurant
4,EmporeÃ­o Emborio,36.358056,25.446111,Desiderio,36.356713,25.445836,Café


Check the unique categories

In [22]:
print('There are {} uniques categories.'.format(len(place_venues['Venue Category'].unique())))

There are 260 uniques categories.


We then one hot the venue’s category for each venues, group them by the neighborhood and calculate the means for each category. 
For each neighborhood we get the top 10 most common venues by sorting the mean of category in descending order.

In [23]:
# one hot encoding
place_onehot = pd.get_dummies(place_venues[['Venue Category']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
place_onehot['Neighborhood'] = place_venues['Neighborhood'] 



# move neighborhood column to the first column
fixed_columns = [place_onehot.columns[-1]] + list(place_onehot.columns[:-1])
place_onehot = place_onehot[fixed_columns]

place_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bathing Area,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Café,Camera Store,Candy Store,Casino,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Dive Spot,Dog Run,Donut Shop,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Taverna,Fishing Spot,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kafenio,Korean Restaurant,Lake,Lighting Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outdoors & Recreation,Ouzeri,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shrine,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Som Tum Restaurant,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tour Provider,Trail,Travel & Transport,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,EmporeÃ­o Emborio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group by neighborhoods

In [24]:
place_grouped = place_onehot.groupby('Neighborhood').mean().reset_index()
place_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bathing Area,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Café,Camera Store,Candy Store,Casino,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cruise,Cultural Center,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Dive Spot,Dog Run,Donut Shop,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Taverna,Fishing Spot,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kafenio,Korean Restaurant,Lake,Lighting Store,Liquor Store,Lounge,Magirio,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outdoors & Recreation,Ouzeri,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shrine,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Som Tum Restaurant,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tour Provider,Trail,Travel & Transport,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,AdejeAdeche,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.

In [25]:
place_grouped.shape

(218, 260)

Define function to return most common venues

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Get top 10 most common venues for neighborhoods

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = place_grouped['Neighborhood']

for ind in np.arange(place_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(place_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AdejeAdeche,Pizza Place,Spanish Restaurant,Italian Restaurant,Coffee Shop,Café,Other Great Outdoors,Fried Chicken Joint,Lounge,Plaza,Gym / Fitness Center
1,Agia Paraskevi,Hotel,Bed & Breakfast,Convenience Store,Fish Taverna,Greek Restaurant,Hotel Pool,Cultural Center,Event Service,Creperie,Fishing Spot
2,Agios GeorgiosAgios Georgios,Hotel,Beach Bar,Fish Taverna,Greek Restaurant,Café,Resort,Lounge,Restaurant,Beach,Bed & Breakfast
3,AguloAgulo,Spanish Restaurant,Scenic Lookout,Tapas Restaurant,Restaurant,Farm,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Women's Store
4,AgÃ­a EirÃ­niAgia Eirini,Greek Restaurant,Harbor / Marina,Fishing Spot,Fish Taverna,Fish & Chips Shop,Fireworks Store,Financial or Legal Service,Filipino Restaurant,Donut Shop,Fast Food Restaurant


Run k-means clusters

Upon the resulted dataframe, we run k means clustering by using clustering number 5, add clustering labels for neighborhoods in the new dataframe. 

In [28]:
# set number of clusters
kclusters = 5

place_grouped_clustering = place_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(place_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 4, 1, 0, 4, 2, 1, 4], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,AdejeAdeche,Pizza Place,Spanish Restaurant,Italian Restaurant,Coffee Shop,Café,Other Great Outdoors,Fried Chicken Joint,Lounge,Plaza,Gym / Fitness Center
1,0,Agia Paraskevi,Hotel,Bed & Breakfast,Convenience Store,Fish Taverna,Greek Restaurant,Hotel Pool,Cultural Center,Event Service,Creperie,Fishing Spot
2,0,Agios GeorgiosAgios Georgios,Hotel,Beach Bar,Fish Taverna,Greek Restaurant,Café,Resort,Lounge,Restaurant,Beach,Bed & Breakfast
3,4,AguloAgulo,Spanish Restaurant,Scenic Lookout,Tapas Restaurant,Restaurant,Farm,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Women's Store
4,1,AgÃ­a EirÃ­niAgia Eirini,Greek Restaurant,Harbor / Marina,Fishing Spot,Fish Taverna,Fish & Chips Shop,Fireworks Store,Financial or Legal Service,Filipino Restaurant,Donut Shop,Fast Food Restaurant
5,0,AkrotÃ­ri Akrotiri,Hotel,Greek Restaurant,Museum,Athletics & Sports,Café,Beach,Supermarket,Historic Site,Taverna,Filipino Restaurant
6,4,ArafoArafo,Construction & Landscaping,Pool,Breakfast Spot,Plaza,Historic Site,Fish Taverna,Fireworks Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant
7,2,AronaAron,Diner,Boarding House,Restaurant,Café,Women's Store,Event Service,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
8,1,Ban Ao KungBan Ao Kung,Harbor / Marina,Women's Store,Duty-free Shop,Fishing Spot,Fish Taverna,Fish & Chips Shop,Fireworks Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant
9,4,Ban Ao MakhamBan Ao Makham,Auto Workshop,Women's Store,Flea Market,Fishing Spot,Fish Taverna,Fish & Chips Shop,Fireworks Store,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant


In [30]:
place_merged = place_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
place_merged = pd.merge(place_merged.dropna(), neighborhoods_venues_sorted.dropna(), how='right', on='Neighborhood')
place_merged # check the last columns!

,Neighborhood,Country,Feature class,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EmporeÃ­o Emborio,"Greece, South AegeanKyklÃ¡des > Santorini","populated placepopulation 1,946",36.358056,25.446111,2,Café,Grilled Meat Restaurant,Bus Stop,Castle,Grocery Store,Plaza,Department Store,Dessert Shop,Fish Taverna,Fish & Chips Shop
1,OÃ­a Apano Meria,"Greece, South AegeanKyklÃ¡des > Santorini","populated placepopulation 3,376",36.462500,25.376111,0,Hotel,Greek Restaurant,Café,Restaurant,Mediterranean Restaurant,Vacation Rental,Wine Bar,Cocktail Bar,Spa,Modern Greek Restaurant
2,Fira FIRA,"Greece, South AegeanKyklÃ¡des > Santorini",seat of a third-order administrative divisionp...,36.420833,25.430833,0,Hotel,Café,Greek Restaurant,Souvlaki Shop,Bar,Taverna,Restaurant,Lounge,Mediterranean Restaurant,Nightclub
3,MessariaMesaria,"Greece, South AegeanKyklÃ¡des > Santorini","populated placepopulation 1,233",36.398889,25.448056,0,Hotel,Gym / Fitness Center,Flower Shop,Soccer Field,Supermarket,Electronics Store,Bakery,Greek Restaurant,Café,Souvlaki Shop
4,MegalochoriMegalochori,"Greece, South AegeanKyklÃ¡des > Santorini",populated place,36.375833,25.431667,0,Hotel,Greek Restaurant,Bakery,Resort,Grilled Meat Restaurant,Meze Restaurant,Bed & Breakfast,Taverna,Restaurant,Winery
5,Therasia Manolas,"Greece, South AegeanKyklÃ¡des > Santorini",populated placepopulation 160,36.434722,25.344722,4,Greek Restaurant,Taverna,Bay,Fast Food Restaurant,Electronics Store,Event Service,Falafel Restaurant,Farm,Farmers Market,Women's Store
6,Imerovigli Imerovigli,"Greece, South AegeanKyklÃ¡des > Santorini",populated place,36.432778,25.422778,0,Hotel,Greek Restaurant,Café,Resort,Steakhouse,Beach Bar,Bed & Breakfast,Souvlaki Shop,Restaurant,Castle
7,EpiskopÃ­ GoniÃ¡s Episkopi Ghonias,"Greece, South AegeanKyklÃ¡des > Santorini",populated place,36.383333,25.460833,4,Winery,Cruise,Brewery,Art Gallery,Women's Store,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
8,AkrotÃ­ri Akrotiri,"Greece, South AegeanKyklÃ¡des > Santorini",populated place,36.357778,25.397500,0,Hotel,Greek Restaurant,Museum,Athletics & Sports,Café,Beach,Supermarket,Historic Site,Taverna,Filipino Restaurant
9,PÃ½rgos Pirghos,"Greece, South AegeanKyklÃ¡des > Santorini",populated place,36.382778,25.449722,0,Hotel,Café,Greek Restaurant,Mediterranean Restaurant,Taverna,Modern Greek Restaurant,Snack Place,Bakery,Kafenio,Souvenir Shop


## Results and Discussion <a name="results"></a>

Show the clustering result in the map

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(place_merged['Latitude'], place_merged['Longitude'], place_merged['Neighborhood'], place_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Then we group again the dataframe by the place name and sum the number of neighborhoods for each of clustering labels. 

In [32]:
def getClusterLabelCount(place, label):
    return place_merged[place_merged.Country.str.contains(place)][place_merged[place_merged.Country.str.contains(place)]['Cluster Labels'] == label].shape[0]

We got the dataframe of number of neighnohoods for each cluster lable as

In [51]:
dfPlace = pd.DataFrame()
for (place) in ['Tenerife', 'Santorini', 'Maldives', 'Phuket', 'Honolulu']:
    clusters = []
    for label in list(range(0,5)):
        clusters.append(getClusterLabelCount(place, label))
    dfPlace[place] = clusters

dfPlace = dfPlace.T
dfPlace.columns = ['neighbohood cluster 0', 'neighbohood cluster 1', 'neighbohood cluster 2', 'neighbohood cluster 3', 'neighbohood cluster 4']
dfPlace

,neighbohood cluster 0,neighbohood cluster 1,neighbohood cluster 2,neighbohood cluster 3,neighbohood cluster 4
Tenerife,9,0,7,0,29
Santorini,23,2,1,2,5
Maldives,5,2,23,5,9
Phuket,5,2,0,2,39
Honolulu,1,1,2,5,39


For Tenerife, we got 9, 0, 7, 0, 29 for lables 0, 1, 2, 3, 4. It means in Tenerife, there are 9 neighborhoods under cluster 0, 7 under cluster 2 and 29 under cluster 4. 0 under cluster 1 and 3.

We again run the k-means clustering against the new dataframe by using group 4.

Norminize the data

In [48]:
from sklearn.preprocessing import StandardScaler

X = dfPlace.values[:,:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.05241424, -1.75      ,  0.04680364, -1.44398974,  0.32954102],
       [ 1.88691271,  0.75      , -0.65525093, -0.4125685 , -1.31816409],
       [-0.47172818,  0.75      ,  1.91894915,  1.13456337, -1.04354657],
       [-0.47172818,  0.75      , -0.77226002, -0.4125685 ,  1.01608482],
       [-0.99587059, -0.5       , -0.53824183,  1.13456337,  1.01608482]])

Define function for running k-means clustering for places

In [56]:
def labelPlace(kclusters, dfPlace):
# set number of clusters
    dfPlace_grouped_clustering = dfPlace

# run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfPlace_grouped_clustering)

# check cluster labels generated for each row in the dataframe
    kmeans.labels_[0:5] 
    result = dfPlace.copy()
    result['label'] = kmeans.labels_[0:5] 
    return result

After running again the k-means clustering against this dataframe by using cluster number as 4, we got

In [60]:
result = labelPlace(4, dfPlace)
result

,neighbohood cluster 0,neighbohood cluster 1,neighbohood cluster 2,neighbohood cluster 3,neighbohood cluster 4,label
Tenerife,9,0,7,0,29,3
Santorini,23,2,1,2,5,2
Maldives,5,2,23,5,9,1
Phuket,5,2,0,2,39,0
Honolulu,1,1,2,5,39,0


### Discussion

There are many of facts that can affect the results. E.g. as we get the neighborhoods for each places from https://www.geonames.org/ by search the feature class as “city, village” and limit the neighborhood as first 50 sorted by population, we might lost some popular venues because the neighborhood is not in the result. And also search the venues for each of neighborhood by specifying the radius as 5 km, some of the actual famous venues, sight, places may not be contained in the result e.g. the mountains that is not within 5km of any neighborhoods. In addition, we rely on the foursquare API to get the features of each place, this can create bias since restaurant, hotels may always be the most common places that the user will visit. Last, we run the k-means clustering against the Dataframe-1, which only contains 5 places as an example. By using the cluster number 4 where the places number is 5. Changing the number of clusters will affect the grouping. E.g. if we use 3 as cluster number

In [61]:
result = labelPlace(3, dfPlace)
result

,neighbohood cluster 0,neighbohood cluster 1,neighbohood cluster 2,neighbohood cluster 3,neighbohood cluster 4,label
Tenerife,9,0,7,0,29,0
Santorini,23,2,1,2,5,2
Maldives,5,2,23,5,9,1
Phuket,5,2,0,2,39,0
Honolulu,1,1,2,5,39,0


Phuket and Honolulu will be still in the same label, which is the same as using cluster number 4. Tenerife is under also under label 0, which seems inline with the fact that it has also most neighborhoods under cluster 4.

If using cluster number as 2

In [62]:
result = labelPlace(2, dfPlace)
result

,neighbohood cluster 0,neighbohood cluster 1,neighbohood cluster 2,neighbohood cluster 3,neighbohood cluster 4,label
Tenerife,9,0,7,0,29,0
Santorini,23,2,1,2,5,1
Maldives,5,2,23,5,9,1
Phuket,5,2,0,2,39,0
Honolulu,1,1,2,5,39,0


The Santorini and Maldives are in one group while Tenerife, Phuket and Honolulu are in the another. 

## Conclusion <a name="conclusion"></a>

In this project, we demonstrate how to compare the places by analyzing their neighborhood information. 
We used FourSquare APIs to get venues information of neighborhoods. 
K-means clustering is used to clustering the neighborhoods and the places.  The results shows the grouping the places are inline with the clustering the neighborhoods of each places. In our example, Phuket and Honolulu are similar than rest of three places. Tenerife is similar to Phuket and Honolulu, than Maldives and Santorini.